In [ ]:
import pandas as pd
import json

# Load CSV file
df_carbon_intensity = pd.read_json("../data/carbonintensitydata.json").T



## Data loading

In [ ]:
df_carbon_intensity

In [ ]:
df_devices = pd.read_json("../data/devices_generated.json")
df_devices = pd.json_normalize(df_devices["devices"])

In [ ]:
df_devices

In [ ]:
df_time_seriesV2 = pd.read_json("../data/timeseries.json") 

In [ ]:
df_time_seriesV2

In [ ]:
inner = df_time_seriesV2["hardware_usages"].map(
    lambda d: next(iter(d.values())) if isinstance(d, dict) and d else {}
)

# 2) Expand to columns (device IDs) and index by time
df_matrix = inner.apply(pd.Series)
df_matrix.index = pd.to_datetime(df_time_seriesV2["time"])  # optional but handy
df_hardware_usage = df_matrix.sort_index().sort_index(axis=1)

In [ ]:
df_hardware_usage

In [ ]:
inner = df_time_seriesV2["pue_values"].map(
    lambda d: next(iter(d.values())) if isinstance(d, dict) and d else {}
)

# 2) Expand to columns (device IDs) and index by time
df_matrix = inner.apply(pd.Series)
df_matrix.index = pd.to_datetime(df_time_seriesV2["time"])  # optional but handy
df_pue = df_matrix.sort_index().sort_index(axis=1)

In [ ]:
df_pue

## SCI computation

### Compute O

First, we do a test for a specific device

In [ ]:
df_carbon_intensity[df_devices[df_devices["device_id"]=="0000000000"]["location"]]

In [ ]:
# start with the O

In [ ]:
device_id = "0000000000"



In [ ]:
tmp_pue = df_pue[device_id]

In [ ]:
tmp_carbon = df_carbon_intensity[df_devices[df_devices["device_id"]==device_id]["location"]]

In [ ]:
power_idle = df_devices[df_devices["device_id"]==device_id]["power_idle"].values[0]

In [ ]:
power_var = df_devices[df_devices["device_id"]==device_id]["power_variable"]

In [ ]:
tmp_usage = df_hardware_usage[device_id] 

In [ ]:
power_idle

In [ ]:
tmp_carbon.columns = ["carbon"]

In [ ]:
df_o = pd.concat([tmp_pue.rename("pue"), tmp_carbon], axis=1)
df_o = pd.concat([df_o, tmp_usage.rename("usage")], axis=1)
df_o["idle"] = power_idle
df_o["power"] = power_var

In [ ]:
df_o

In [ ]:
df_o["energy_consumption"] = ((df_o["power"] * df_o["usage"] ) + df_o["idle"]) *0.5

In [ ]:
df_o

In [ ]:
df_o.drop(["idle", "power", "usage"], inplace=True, axis=1)

In [ ]:
df_o

In [ ]:
df_o["o"] = (df_o["pue"] * df_o["carbon"] * df_o["energy_consumption"]) /1000

In [ ]:
df_o.rename({"o":"test"}, axis=1)

Then we compute O for the all system, i.e. all devices

In [ ]:
df_final_o = pd.DataFrame()

for device_id in df_devices.device_id:
    #get the inputs
    tmp_pue = df_pue[device_id]
    tmp_carbon = df_carbon_intensity[df_devices[df_devices["device_id"]==device_id]["location"]]
    power_idle = df_devices[df_devices["device_id"]==device_id]["power_idle"].values[0]
    power_var = df_devices[df_devices["device_id"]==device_id]["power_variable"].values[0]
    tmp_usage = df_hardware_usage[device_id] 

    #combine them into a DF
    tmp_carbon.columns = ["carbon"]
    df_o = pd.concat([tmp_pue.rename("pue"), tmp_carbon], axis=1)
    df_o = pd.concat([df_o, tmp_usage.rename("usage")], axis=1)
    df_o["idle"] = power_idle
    df_o["power"] = power_var

    #make the computations
    df_o["energy_consumption"] = ((df_o["power"] * df_o["usage"] ) + df_o["idle"]) *0.5
    df_o.drop(["idle", "power", "usage"], inplace=True, axis=1)
    df_o["o"] = (df_o["pue"] * df_o["carbon"] * df_o["energy_consumption"]) /1000

    #save everything
    df_final_o = pd.concat([df_final_o, df_o.rename({"o":device_id}, axis=1)[device_id]], axis=1)

# add functional units 
df_functinal_units = df_time_seriesV2["functional_units"]
df_functinal_units.index = df_time_seriesV2["time"]
df_final_o.index = pd.to_datetime(df_final_o.index).tz_convert("UTC")
df_functinal_units.index = pd.to_datetime(df_functinal_units.index).tz_convert("UTC")

df_final_o = pd.concat([df_final_o.sum(axis=1), df_functinal_units], axis=1)

# final computation for o
df_final_o["o"] = df_final_o[0] / df_final_o["functional_units"]  
df_final_o.drop([0, "functional_units"], inplace=True, axis=1)

display(df_final_o)

## compute M